# Determine the drag coefficient with DRIA

To construct the orbit arcs we are going to use the `arc_times.txt` file.
- just use the orbit start and stop times
- turn on the DRIA usage: ` 'cd_model'       : {'input': 'DRIA'},
`

### TO DO:
- construct the 30-hour arcs using the provided data arcs



## Read arc_times

The below snippet reads arc times from the `arc_times.txt` file and uses them as the inputs for subsequent runs.
The scaling factor cadence is based on trimming the excess ~<3 hours off the ends and having a time stamp every 3-hours

In [1]:
# test_index = 12
# test_index = 24
scaling_cadence = 1.5
scale_cadence = scaling_cadence


In [2]:
import pandas as pd
from datetime import datetime,timedelta
import numpy as np

arc_timesfile = '/data/SatDragModelValidation/data/inputs/sat_icesat2/arc_times.txt'


### Make a function that finds the right range of dates in this file.
arcs = pd.read_csv(arc_timesfile, 
            sep = ',',
#             dtype=object,
            names = [
                'arc'         ,
                'epoch_start' ,
                'epoch_stop'  ,
                'orbit_start' ,
                'orbit_stop'  ,
                    ],)


arcs_yyyyddd = [x.strip() for x in arcs['arc'].values.tolist()]
epochstart   = [x.strip() for x in arcs['orbit_start'].values.tolist()]
epochstop    = [x.strip() for x in arcs['orbit_stop' ].values.tolist()]


# month_list = ['oct', 'nov', 'dec']
month_list = ['nov']
for imonth, month in enumerate(month_list):
    if month=='oct':
        m_num = 10
    if month=='nov':
        m_num = 11
    if month=='dec':
        m_num = 12
    
    input_arcs       = []
    input_epochstart = []
    input_epochstop  = []
    arc_length = []
    scaleparameter_times = []
    
    for i,val in enumerate(arcs_yyyyddd):
        
        if pd.to_datetime(val[:8],format="%Y.%j").month == m_num:
            epoch_delta  =  pd.to_timedelta(pd.to_datetime(epochstop[i]) - pd.to_datetime(epochstart[i]), 'hours')
            arc_length.append(epoch_delta.total_seconds()/3600)

            if val[8:] == 'A' or val[8:] == 'B':
#                 input_arcs.append(      arcs_yyyyddd[i])
#                 input_epochstart.append(epochstart[i])
#                 input_epochstop.append( epochstop[i])
#                 scaleparameter_times.append([''])
                pass
            else:
                ### use 24-hour start/stop
                # input_arcs.append(      arcs_yyyyddd[i])
                # input_epochstart.append(str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j")))
                # input_epochstop.append( str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j")+ pd.to_timedelta(24,'h')))
                
                ### Use Arc text file times
                input_arcs.append(      arcs_yyyyddd[i])
                input_epochstart.append(epochstart[i])
                input_epochstop.append( epochstop[i])
#                 print(f"epoch_top {epochstop[i]}")
                scalestart = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j"))
                scalestop  = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j") + pd.to_timedelta(24,'h'))
                scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                          end   = pd.to_datetime(scalestop ),
                                                          freq=str(scale_cadence)+"H")

                ###append the epoch end time to the end
                add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                              pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)

                
                scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                    format ='%y%m%d%H%M%S.%f' )
                                 for ts in add_epochend ])
    
for i,val in enumerate(input_arcs):
    print(f" {input_arcs[i]:10}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}, {arc_length[i]}")
    if val[8:] == 'A' or val[8:] == 'B':
        print("")

    else:
        print(f" {'':10}  , {scaleparameter_times[i]}")
        print("")    
    


 2018.305    , 2018-10-31 21:18:00 ,  2018-11-02 02:42:00 , 29.4
             , [Timestamp('2018-11-01 00:00:00'), Timestamp('2018-11-01 01:30:00'), Timestamp('2018-11-01 03:00:00'), Timestamp('2018-11-01 04:30:00'), Timestamp('2018-11-01 06:00:00'), Timestamp('2018-11-01 07:30:00'), Timestamp('2018-11-01 09:00:00'), Timestamp('2018-11-01 10:30:00'), Timestamp('2018-11-01 12:00:00'), Timestamp('2018-11-01 13:30:00'), Timestamp('2018-11-01 15:00:00'), Timestamp('2018-11-01 16:30:00'), Timestamp('2018-11-01 18:00:00'), Timestamp('2018-11-01 19:30:00'), Timestamp('2018-11-01 21:00:00'), Timestamp('2018-11-01 22:30:00'), Timestamp('2018-11-02 00:00:00'), Timestamp('2018-11-02 02:42:00')]

 2018.306    , 2018-11-01 21:18:00 ,  2018-11-03 02:42:00 , 29.4
             , [Timestamp('2018-11-02 00:00:00'), Timestamp('2018-11-02 01:30:00'), Timestamp('2018-11-02 03:00:00'), Timestamp('2018-11-02 04:30:00'), Timestamp('2018-11-02 06:00:00'), Timestamp('2018-11-02 07:30:00'), Timestamp('2018-11-02

Select the dates within the month of interest

## Run geodyn

In [3]:
from pygeodyn.PYGEODYN import Pygeodyn
import datetime


index_buffarc_start = 0
run_list = [
                'msis2',
                'jb2008',
                'dtm2020_o',
           ]

month_list = ['nov']
# month_list = ['oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr']




dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    for imonth,month in enumerate(month_list):
        if i =='msis2':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 5
            run_dict[month+i]['model_path'] = None
        if i =='dtm2020_o':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 3
            run_dict[month+i]['model_path'] = None
        if i =='jb2008':
            run_dict[month+i]={}
            run_dict[month+i]['num'] = 1
            run_dict[month+i]['model_path'] = None

print(run_dict)

{'novmsis2': {'num': 5, 'model_path': None}, 'novjb2008': {'num': 1, 'model_path': None}, 'novdtm2020_o': {'num': 3, 'model_path': None}}


In [4]:
# import sys
# sys.exit(0)

In [5]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os
from datetime import datetime,timedelta


g2b_path = "/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"
dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'icesat2/O2R2023_longimeperiod/1_DRIAruns_1.5/'



obj = {}

for imonth,month in enumerate(month_list):
    if month=='oct':
        m_num = 10
    if month=='nov':
        m_num = 11
    if month=='dec':
        m_num = 12
    if month=='jan':
        m_num = 1
    if month=='feb':
        m_num = 2
    if month=='mar':
        m_num = 3
    if month=='apr':
        m_num = 4

#     file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_2018_{m_num:02d}.txt"
#     file_g2b     = f"pce_icesat2_pso_2018_{m_num:02d}"
    file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_2018_{m_num:02d}.txt"
    file_g2b     = f"pce_icesat2_pso_2018_{m_num:02d}"

    input_arcs       = []
    input_epochstart = []
    input_epochstop  = []
    arc_length = []
    scaleparameter_times = []
    
    for i,val in enumerate(arcs_yyyyddd):
        if pd.to_datetime(val[:8],format="%Y.%j").month == m_num:
            epoch_delta  =  pd.to_timedelta(pd.to_datetime(epochstop[i]) - pd.to_datetime(epochstart[i]), 'hours')
            arc_length.append(epoch_delta.total_seconds()/3600)

            if val[8:] == 'A' or val[8:] == 'B':
#                 input_arcs.append(      arcs_yyyyddd[i])
#                 input_epochstart.append(epochstart[i])
#                 input_epochstop.append( epochstop[i])
#                 scaleparameter_times.append([''])
                pass
            else:
                ### use 24-hour start/stop
                # input_arcs.append(      arcs_yyyyddd[i])
                # input_epochstart.append(str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j")))
                # input_epochstop.append( str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j")+ pd.to_timedelta(24,'h')))
                
                ### Use Arc text file times
                input_arcs.append(      arcs_yyyyddd[i])
                input_epochstart.append(epochstart[i])
                input_epochstop.append( epochstop[i])
                scalestart = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j"))
                scalestop  = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j") + pd.to_timedelta(24,'h'))
                scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                          end   = pd.to_datetime(scalestop ),
                                                          freq=str(scale_cadence)+"H")

                ###append the epoch end time to the end
                add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                              pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)
                scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                    format ='%y%m%d%H%M%S.%f' )
                                 for ts in add_epochend ])


      
    if month=='dec':
        input_arcs       = input_arcs[:-1] 
        input_epochstart = input_epochstart[:-1] 
        input_epochstop  = input_epochstop[:-1] 
    
#     for i,val in enumerate(input_arcs):
#         print(f" {input_arcs[i]:10}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}, {arc_length[i]}")
#         if val[8:] == 'A' or val[8:] == 'B':
#             print("")

#         else:
#             print(f" {'':10}  , {scaleparameter_times[i]}")
#             print("")    
    
    for i,den in enumerate(run_list):
        settings_icesat2= {# Basic input settings
                     'satellite'      : {'input': 'icesat2'},
                     'den_model'      : {'input': den},
                     'run_type'       : {'input': 'DataReduction_PCE'},
                     'run_specifier'  : {'input': '_'+month+'scale_'+str(scale_cadence)},
                     'cd_model'       : {'input': 'DRIA'},
                     'file_string'    : {'input': 'DRIAscaled_'+str(scale_cadence)},
                     'model_data_path' : {'input': run_dict[month+den]['model_path']},
                     # Force Model settings
                      'cd_type'               : {'input':"Fixed_CD"},
                      'cd_value'              : {'input':1.0 },
                      'scaling_factor'        : {'input':True},
                      'hours_between_cd_adj'  : {'input':scaling_cadence},
                      #### Comment for unadjusted run:
                      'cd_adjustment_boolean' : {'input':True },
                    #### DRIA CD Model Parameters
                    'cd_model_params' : {'input':{ 
                            'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
                            'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
                               ###  Alpha is b/w 0 and 1
                            'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient
                            'KL'     : '0.0D0'    ,    #!  langmuir parameter
                            'FRACOX' : '1.0D0'   ,     #!  fraction of surface covered by atomic oxygen
                       }},
                      #### ---------------------------------------
                     # Run
                      'arc_type'       : {'input':'Nominal30hr_and_AB'},      
                      'step'           : {'input': 10.},
                      'orbfil_step'    : {'input': 120.},
                      'which_ICfile'   : {'input':file_raw_ICs},
                      'which_g2bfile'  : {'input':file_g2b},
                        #
                      'arc'            : {'input': input_arcs},
                      'epoch_start'    : {'input': input_epochstart},
                      'epoch_stop'     : {'input': input_epochstop},  
                'scaleparameter_times' : {'input': scaleparameter_times},  
                       #                                
                      'global_options' : {'input':'pso_2018'},
                     # Request read on raw outputs
                      'request_data'   : {'input': ['Trajectory_orbfil', 
                                                   'Density', 
                                                   'Residuals_summary',
                                                   'DragFile',
                                                   'AdjustedParams'
                                                   ]},
                  #end dict
                  }


#         sat = Pygeodyn(settings_icesat2, use_file=False)
#         sat.run_arcs()
# #         obj[month+den] =  sat.getData_BigData_lowmemory(orbit_propagation=True)
# #         obj[month+den] = vars(obj[month+den])
#         gc_collect()



        pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

        pickle_file = dir_save+den+pickleName
        if not os.path.exists(pickle_file):
            print('Must create pickle file...')
            print('   ',  pickle_file)
            print('   ', 'Reading Geodyn Data')

            ### Load the data into an object
            sat = Pygeodyn(settings_icesat2, use_file=False)
            obj = sat.getData_BigData_lowmemory(orbit_propagation=False)
            gc_collect()

            #### Pickle the object to save it
            print('   ', 'Saving pickle')
            filehandler = open(pickle_file, 'wb') 
            pickle.dump(vars(obj), filehandler)
            filehandler.close()
            obj = 0
            print('   ', 'Saved pickle')

obj = {}
for i,model in enumerate(run_list):
    for imonth,month in enumerate(month_list):
        pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

        ### Load the data if the pickles exist
        print()
        print()
        gc_collect()

        pickle_file = dir_save+model+pickleName

        filehandler = open(pickle_file, 'rb') 
        obj[month+model] = pickle.load(filehandler)
        filehandler.close()
        print('Loaded data from pickle... ',  model)
    
    
### Save space if doing density retrieval
for model in run_dict.keys():
    del obj[model]['OrbitResids']
    del obj[model]['Trajectory_orbfil']
    
gc_collect()

Must create pickle file...
    /data/SatDragModelValidation/data/outputs_clean/icesat2/O2R2023_longimeperiod/1_DRIAruns_1.5/dtm2020_o_nov_DRIA_scale1.5.pkl
    Reading Geodyn Data
Using the ICESat-2 Class
    - Arc: 2018.305
---Reading PCE data and save as global var
    - Arc: 2018.306
    - Arc: 2018.307
    - Arc: 2018.308
    - Arc: 2018.309
    - Arc: 2018.312
    - Arc: 2018.313
    - Arc: 2018.314
    - Arc: 2018.315
    - Arc: 2018.316
    - Arc: 2018.317
    - Arc: 2018.318
    - Arc: 2018.319
    - Arc: 2018.320
    - Arc: 2018.321
    - Arc: 2018.322
    - Arc: 2018.323
    - Arc: 2018.324
    - Arc: 2018.325
    - Arc: 2018.326
    - Arc: 2018.327
    - Arc: 2018.328
    - Arc: 2018.330
    - Arc: 2018.331
    - Arc: 2018.334
deleting PCE_data_raw from globals. Allows reset for new month file.
    Saving pickle
    Saved pickle


Loaded data from pickle...  msis2


Loaded data from pickle...  jb2008


Loaded data from pickle...  dtm2020_o


0

### geodyn output

## Plot Settings:


In [6]:
from pygeodyn.pygeodyn_plot_scalingfactors import *

coldict = {}
coldict['msis2']     = "#2ca02c"  # 'tab:green'
coldict['dtm2020_o'] = "#d62728"  # 'tab:red'
coldict['jb2008']    = "orange"   
coldict['hasdm_oc']  = "#1f77b4"     





## Plot densities


### Plot Kp

In [7]:
# %load_ext autoreload
# %autoreload 2

# from pygeodyn.pygeodyn_plot_scalingfactors import *
# import os
# import pandas as pd
# from netCDF4 import Dataset
# def read_nc_file( filename, variables):
#     ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
#         The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
#     status = os.path.exists(filename)
    
#     if status == True:
#         data = {}
#         for i, var_names in enumerate(variables):
#             ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
#             varData = ncid.variables
#             data[var_names] = np.array(varData[var_names])  
#     elif status == False:
#         print('No File Found', filename )
#         breakloop = True
#         data = 0
#         return( data , breakloop)
#     breakloop = False
#     return(data,breakloop )


# arc_list = []

# arc_list_18 = np.arange(280,366)
# for i in arc_list_18:
#     val = '2018'+str(i)
#     arc_list.append(int(val))
    
#     #     print(val)
    
# arc_list_19 = np.arange(1,112)
# for i in arc_list_19:
#     val = f"2019{i:03d}"
#     arc_list.append(int(val))


# path_to_f107 = '/data/SatDragModelValidation/data/inputs/atmos_models/geo_phys_indicies/gpi_1960001-2021243_f107aDaily.nc'

# f107_data = read_nc_file(path_to_f107, ['year_day', 'f107d', 'f107a', 'kp'])

# date = []
# kp_list = []
# f107d_list = []
# f107a_list  = []
# date_3hr = []
# doy_list    = []



# for i,val in enumerate(arc_list):
    
#     index = f107_data[0]['year_day']==val
#     kp_list.append(f107_data[0]['kp'][index][0])
#     f107d_list.append(f107_data[0]['f107d'][index][0])
#     f107a_list.append(f107_data[0]['f107a'][index][0])
#     doy_list.append(str(f107_data[0]['year_day'][index][0])[-3:])

#     date.append(pd.to_datetime( str(val), format='%Y%j'))

#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=0))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=3))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=6))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=9))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=12))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=15))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=18))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=21))
# #     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=24))
    
# kp_expand = []
# for i in kp_list:
#     for ii in i:
#         kp_expand.append(ii)
        
        
        
# solar_fluxes = {}
# solar_fluxes['f107d_list'] = f107d_list
# solar_fluxes['f107a_list'] = f107a_list
# solar_fluxes['date']       = date
# solar_fluxes['date_3hr']   = date_3hr
# solar_fluxes['kp_expand']  = kp_expand

# f107d_earth = []
# f107a_earth = []
# ######################################################################### 
# ##### Account for the F10.7 at earth (instead of referenced at 1AU) #####
# ######################################################################### 

# for i_doy,val_doy in enumerate(doy_list):
#     iday = int(val_doy)
#     theta0 = 2 * np.pi * (iday)/365.
#     sfeps = 1.000110 + 0.034221*np.cos(theta0)+0.001280* np.sin(theta0) +0.000719*np.cos(2.*theta0)+0.000077*np.sin(2.*theta0)

#     f107d_earth.append(sfeps * solar_fluxes['f107d_list'][i_doy])
#     f107a_earth.append(sfeps * solar_fluxes['f107a_list'][i_doy])

# solar_fluxes['f107d_earth'] = f107d_earth
# solar_fluxes['f107a_earth'] = f107a_earth





# fig = make_subplots(rows=2, cols=1,
#                     vertical_spacing = 0.05,
#                     shared_xaxes=True)


# index1 = 21
# index2 = 37
# index3h_1 = 168
# index3h_2 = 289

# fig.add_trace(go.Scatter(x=solar_fluxes['date'],
#                            y=solar_fluxes['f107d_earth'],
#                            name= 'F107d_1AU',
#                            mode='lines',
#                            opacity=1,
# #                                marker=dict(color='cornflowerblue', size=2 ),
#                            line = dict(shape = 'hvh',dash='dash', color = 'black', width=2),
#                            showlegend=False),
#                            secondary_y=False,row=1, col=1)
# fig.add_trace(go.Scatter(x=solar_fluxes['date'],
#                            y=solar_fluxes['f107a_earth'],
#                            name= 'F107a_1AU',
#                            mode='lines',
#                            opacity=1,
# #                                marker=dict(color='cornflowerblue', size=2 ),
#                            line = dict(shape = 'hvh', color = 'black', width=2),
#                            showlegend=False),
#                            secondary_y=False,row=1, col=1)

# fig.add_trace(go.Scatter(x=solar_fluxes['date_3hr'],
#                            y=solar_fluxes['kp_expand'],
#                            name= 'Kp',
#                            mode='lines',
#                            opacity=1,
# #                                marker=dict(color='black',size=2),
#                            line = dict(shape = 'hvh', color = 'black', width=2),
#                            showlegend=False),
#                            secondary_y=False,row=2, col=1) 




    
# #######################################################
# font_dict=dict(family='Arial',size=16,color='black')
# #######################################################

# fig.update_yaxes(title_text=r"$\text{F}_{\text{10.7}}\text{ Solar Flux (sfu)}$", 
# #                  color='black',
# #                  range=[64.5, 86],
#                   row=1, col=1,)

# fig.update_yaxes(title_text=r"$\text{K}_\text{p}\text{ Index}$",
# #                   range=[-0.3, 8],
#                  row=2, col=1,)

   
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)


# fig.update_layout(autosize=False,    width=1000,    height=500,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict, plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=14)  # Increase size of subplot title
# fig.show(#renderer="jpg",
#          config=config)

# ### pio.write_image(fig, plots_dir+'twoweek_fullresult.pdf')
# # pio.write_image(fig, plots_dir+'f107_kp_twoweek.jpg', scale=5)


In [8]:
# obj[model]['Statistics']#['T_RMS'].values[0]

### original scaling (no ensemble)

In [9]:
# %load_ext autoreload
# %autoreload 2



# satid = 1807001
# for plot_num, model in enumerate(run_list):
#     for imonth,month in enumerate(month_list):
#         print(model,month)

    
#         ScalingFactors      = []
#         ScalingFactor_times = []
#         PercChange          = []

#         for ii,arc in enumerate(obj[month+model]['global_params']['arc_input']):
# #             print()
#             epochstart = obj[month+model]['global_params']['prms']['epoch_start'][ii]
#             hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#             frachours =(hrs/24)
#             #
#             if len(arc) == 9:
#                 maneuv_indicator = arc[8]
#             else:
#                 maneuv_indicator = ''
#             arc_type = obj[month+model]['global_params']['prms']['arc_type']
#             if arc_type == "Nominal30hr_and_AB":
#                 arc_name =arc[:8]+ maneuv_indicator
#             else:
#                 arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator


#             iters = int(obj[month+model]['run_parameters'+arc_name]['total_iterations'])
#             for iit, itime in enumerate(obj[month+model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
#                 if iit == 0 or iit==9:
#                     pass
#                 else:
#                     CURRENT_VALUE = obj[month+model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
#                     APRIORI_VALUE = obj[month+model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
#                     ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
#                     ScalingFactor_times.append(itime) 
#                     PercChange.append(calc_percent_change(APRIORI_VALUE, CURRENT_VALUE))
#     #                 print(itime,CURRENT_VALUE )

#         run_dict[month+model]['ScalingFactor_times'] = ScalingFactor_times
#         run_dict[month+model]['ScalingFactors']      = ScalingFactors
#         run_dict[month+model]['percent_change']      = PercChange

    
    
# #### SCALE THE DENSITIES:

# models_dens = {}
# for monthmodel in run_dict.keys():
#     print(model)
#     models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

    
        


# fixing scaling with ensemble

In [10]:

# def calc_rho_ScaledEnsembleWgtAvg(OBJECT, model_dict, model_fill, scale_cadence):
    
#     """REDO THIS USING THE SCALED DENSITY DICT FROM THE ABOVE"""


#     dates   = []
#     rho_scaledEnsembleAvg = []
#     std_scaledEnsembleAvg = []
#     list_lat = []

#     # i_countfactor = 0

#     # for ii,arc in enumerate(OBJECT[model_fill]['global_params']['arc_input']):

#     #     epochstart = OBJECT[model_fill]['global_params']['prms']['epoch_start'][ii]
#     #     epochstop  = OBJECT[model_fill]['global_params']['prms']['epoch_stop'][ii]
#     #     hrs        = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#     #     frachours  =(hrs/24)
#     #     #
#     #     arc_name =arc+('%.3f'%frachours).lstrip('0')

#     for ii,arc in enumerate(OBJECT[model_fill]['global_params']['arc_input']):
#         epochstart = OBJECT[model_fill]['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = OBJECT[model_fill]['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator

#         start_arc = OBJECT[model_fill]['run_parameters'+arc_name]['prms_arc']['scaleparameter_times'][0]
#         end_arc   = OBJECT[model_fill]['run_parameters'+arc_name]['prms_arc']['scaleparameter_times'][-1]

#         #
#         # start_arc  = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S') + pd.to_timedelta(scale_cadence,'h')
#         # end_arc   = pd.to_datetime(epochstop, format='%Y-%m-%d %H:%M:%S') - pd.to_timedelta(scale_cadence,'h')

        
# #         epochstart = OBJECT[model_fill]['global_params']['prms']['epoch_start'][ii]
# #         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
# #         frachours =(hrs/24)
        
# #         arc_name =arc+('%.3f'%frachours).lstrip('0')
# #         start_arc = pd.to_datetime(arc, format='%Y.%j')
# #         end_arc = pd.to_datetime(arc, format='%Y.%j')+ pd.to_timedelta(24,'h')
# #         print('arc',arc)
# #         print('start_arc', start_arc)
# #         print('end_arc', end_arc)
# #         print()
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         A = OBJECT[model_fill]['Density'][arc_name].query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )
#         len_dates = np.shape(A['rho (kg/m**3)'])[0]

#         ### Loop thru all dates of interest in this arc
#         ###   for each date we will make a list of rho, lats, wgts for each model
#         for it in np.arange(0,len_dates):
#             list_it_models = []
#             list_it_lat    = []
#             wgts           = []


#             for model in model_dict.keys():
#                 # print(A)
#                 A = OBJECT[model]['Density'][arc_name].query( \
#                     f"{start_arc.year}"         \
#                    +f"{start_arc.month:02d}"    \
#                    +f"{start_arc.day:02d}"      \
#                    +f"{start_arc.hour:02d}"     \
#                    +f"{start_arc.minute:02d}"   \
#                    +f"{start_arc.second:02d}"   \
#                    +f" <= Date < "                     \
#                    +f"{end_arc.year}"       \
#                    +f"{end_arc.month:02d}"  \
#                    +f"{end_arc.day:02d}"    \
#                    +f"{end_arc.hour:02d}"   \
#                    +f"{end_arc.minute:02d}" \
#                    +f"{end_arc.second:02d}" \
#                 )

                
#                 if scale_cadence==24:
#                     ## the 24-hour scaling uses the arc index to index the scaling factor since they have the same cadence
#                     try:
#                         list_it_models.append(A['rho (kg/m**3)'].iloc[it] \
#                                             *model_dict[model]['ScalingFactors'][ii])
#                         list_it_lat.append(A['Lat'].iloc[it])
#                         wgts.append(model_dict[model]['Weight'][arc_name])
#                     except:
#                         continue
#             if scale_cadence ==24: 
#                 dates.append(A['Date'].iloc[it])
#                 list_lat.append(np.average(list_it_lat))
#             else:
                
#             # i_countfactor  = i_countfactor + 1

#             # print(f"  {arc} ")

#             # print("list_it_models",list_it_models)
#             # print("wgts",wgts)

#             # weights are done on an arc-basis and do not depend on Scale Cadence
#             x_bar_wgt  = np.average(list_it_models, weights=wgts)
#             M = np.count_nonzero(wgts)
#             rho_scaledEnsembleAvg.append(  x_bar_wgt   )    
# #             std_scaledEnsembleAvg.append(  np.std(list_it_models)       )
#             std_numerator  = sum([ wgts[i]*((list_it_models[i] - x_bar_wgt)**2) \
#                                                  for i in range(len(wgts)) ])
#             std_denominator= ((M-1)/M)*sum(wgts)
#             std_scaledEnsembleAvg.append(  np.sqrt( std_numerator/std_denominator) )

#     return(dates, list_lat, rho_scaledEnsembleAvg, std_scaledEnsembleAvg)



In [11]:
# obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD']
# obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']

In [12]:
# for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
#     print(iit,itime)

In [13]:
  
satid = 1807001
wgts = {}

for model in run_dict.keys():
    wgts[model] = {}
    ScalingFactors  = []
    ScalingFactor_times = []

    for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
        epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
        hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
        frachours =(hrs/24)
        #
        if len(arc) == 9:
            maneuv_indicator = arc[8]
        else:
            maneuv_indicator = ''
        arc_type = obj[model]['global_params']['prms']['arc_type']
        if arc_type == "Nominal30hr_and_AB":
            arc_name =arc[:8]+ maneuv_indicator
        else:
            arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        ### Collect the weights for the ensemble average
        inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
        wgts[model][arc_name] = inv_rms#/sum_wgts

        iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
        for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
            if iit == 0 or iit==17:
                pass
            else:
                CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                ScalingFactor_times.append(itime)
    run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
    run_dict[model]['ScalingFactors']      = ScalingFactors
    run_dict[model]['Weight'] = wgts[model]
    

    
###### SCALE THE DENSITIES:
models_dens = {}
for monthmodel in run_dict.keys():
    print(f"---Making continuous scaled rho for {monthmodel}")
    models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)


## Retrieve scaled ensemble weighted average
##     'Rho_x' denotes the ensemble weighted avg
models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)


---Making continuous scaled rho for novmsis2
---Making continuous scaled rho for novjb2008
---Making continuous scaled rho for novdtm2020_o
Processing nov


In [16]:
%load_ext autoreload
%autoreload 2


def PLOT_retrievedRHO(fig, obj_m1, den_dict, model_dict):
        
    model_m1 = obj_m1['global_params']['prms']['den_model']

    fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
                               y=model_dict['ScalingFactors'],
                               name= model,
                               mode='markers',
                               opacity=1,
#                                    marker=dict(color=coldict[model_m1], 
#                                                size=5,
#                                                symbol='line-ew',
#                                                line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
                                   marker=dict(color=coldict[model_m1], size=5),
                               line = dict(shape = 'hvh',dash ='dot', color = coldict[model_m1], width=1),
                               showlegend=False),
                               secondary_y=False,row=1, col=1)

    
    time_avg,dscale_avg = orbit_avg_generic(den_dict['datescaled'], den_dict['denscaled'], den_dict['lat'])
    ## -----------------------------------------------------------------------------------------------------
    ##     Orbit Averaged Density
    fig.add_trace(go.Scattergl(x=time_avg,
                               y=dscale_avg,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                                   marker=dict(color=coldict[model_m1],size=2),
                                   line = dict(dash ='solid', color = coldict[model_m1], width=2),
                               showlegend=False), row=2, col=1)
    
    time_avg,d_avg = orbit_avg_generic(den_dict['datescaled'], den_dict['dens'], den_dict['lat'])
    ## -----------------------------------------------------------------------------------------------------

    ##     Orbit Averaged Density
    fig.add_trace(go.Scattergl(x=time_avg,
                               y=d_avg,
                               ### name= model_m1,
                               mode='markers',
                               opacity=0.5,
                                   marker=dict(color= coldict[model_m1],size=3),
                                   line = dict(dash ='solid', color = coldict[model_m1], width=2),
                               showlegend=False), row=2, col=1)
    
    
    
             
    return(fig)







fig  = make_subplots(
    rows=2, cols=1,
    vertical_spacing = 0.05,
    shared_xaxes=True)



# run_dict = { }
# rms_total_return = {}

for model in run_dict.keys():
#     for imonth,month in enumerate(month_list):

    fig = PLOT_retrievedRHO(fig, obj[model],   models_dens[model], run_dict[model])


        

time_avg,Rho_x = orbit_avg_generic(models_dens['novRho_x']['date'],
                                        models_dens['novRho_x']['Rho_x'],
                                        models_dens['novRho_x']['lat'])

time_avg,Rho_std = orbit_avg_generic(models_dens['novRho_x']['date'],
                                        models_dens['novRho_x']['Rho_std'],
                                        models_dens['novRho_x']['lat'])


fig.add_trace(go.Scattergl(x=time_avg,
                           y=Rho_x,
                           ### name= model_m1,
                           mode='markers+lines',
                           opacity=1,
                           marker=dict(color='black',size=5),
                               line = dict( dash ='solid', color = 'black', width=4),
                           showlegend=False), row=2, col=1)
## ERRROR BARS
fig.add_trace(go.Scattergl(x=time_avg,
                           y=np.array(Rho_x)\
                               +np.array(Rho_std),
                           ### name= model_m1,
                           mode='lines',
                           opacity=.8,
                               line = dict( dash ='dash', color = 'grey', width=3),
                           showlegend=False), row=2, col=1)
fig.add_trace(go.Scattergl(x=time_avg,
                           y=np.array(Rho_x)\
                              -np.array(Rho_std),
                           ### name= model_m1,
                           mode='lines',
                           opacity=.8,
                               line = dict( dash ='dash', color = 'grey', width=3),
                           showlegend=False), row=2, col=1)

    
    
#######################################################
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = True
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)

    fig.update_yaxes(title_text="Scaling Factor", 
                     exponentformat= 'power',row=1, col=1)

#     yaxis_range = [-13.7 -.55 ,  -12.6+.25]# ] #full_fig.layout.yaxis2.range

    fig.update_yaxes(title_text="Scaled Density", 
                     type="log", 
#                       range=yaxis_range,
                     exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes(title_text="CD*A", 
# #                       type="log", 
# #                       range=yaxis_range,
#                      exponentformat= 'power',row=3, col=1)
    
a='input'
s=settings_icesat2

fig.update_layout(title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}",
#                     title=f"BWDRAG, 3-hr Scale from CD=2.5 ",
                  autosize=False,    width=900,    height=700,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title
fig.show(config=config)






The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### VIEW PLOT

In [15]:
from sys import exit as sysexit
sysexit()

SystemExit: 

/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
%load_ext autoreload
%autoreload 2


def PLOT_resids(fig, obj_m1, den_dict, model_dict):
    
    SHOW_alldata = False
    
    model_m1 = obj_m1['global_params']['prms']['den_model']
    if model!='hasdm_oc':
        opac_val = 1
    else:
        opac_val = 1

    for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
        epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
        hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
        frachours =(hrs/24)
        #
        if len(arc) == 9:
            maneuv_indicator = arc[8]
        else:
            maneuv_indicator = ''
        arc_type = obj_m1['global_params']['prms']['arc_type']
        if arc_type == "Nominal30hr_and_AB":
            arc_name =arc[:8]+ maneuv_indicator
        else:
            arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        

#         data_res = obj_m1['OrbitResids'][arc_name]['resids']

        drag_data        = obj_m1['DragFile'][arc_name]
        drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]
        
#         print(arc[:8])
        ### Cut off the extra time for the regular days
        start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
        end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

#         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
        ### SELECT ONLY THE MIDDLE 24-HOURS
        drag_data = drag_data.query( \
                f"{start_arc.year}"         \
               +f"{start_arc.month:02d}"    \
               +f"{start_arc.day:02d}"      \
               +f"{start_arc.hour:02d}"     \
               +f"{start_arc.minute:02d}"   \
               +f"{start_arc.second:02d}"   \
               +f" <= Date < "                     \
               +f"{end_arc.year}"       \
               +f"{end_arc.month:02d}"  \
               +f"{end_arc.day:02d}"    \
               +f"{end_arc.hour:02d}"   \
               +f"{end_arc.minute:02d}" \
               +f"{end_arc.second:02d}" \
            )
        ### SELECT ONLY THE MIDDLE 24-HOURS
#         den_df = den_df.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )


        indplot=3

#         date_rms.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) + datetime.timedelta(int(arc.split('.')[1]))- datetime.timedelta(hours=12) ))
#         rms_totals.append(obj_m1['Statistics'][arc]['T_RMS'].values[0])

#         iters = obj_m1['run_parameters'+arc_name]['total_iterations']
#         for itime in obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD']:
#             date_scalefactor.append(itime)
#             ScalingFactor.append(obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD'][itime]['CURRENT_VALUE'])


#         if SHOW_alldata:
#         fig.add_trace(go.Scattergl(x=drag_data['Date'][::5],
#                                    y=drag_data['CD']*drag_data['TOTAREA'][::5],
#                                  mode='markers',
#                                  opacity=1,
#                                  marker=dict(color= coldict[model_m1],size=3,),
#                                  showlegend=False,),
#                                  secondary_y=False,
#                                  row=3, col=1)
        (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD']*drag_data['TOTAREA'],drag_data['Lat'])    
        fig.add_trace(go.Scattergl(x=time_avg,
                                 y=avg_cd,
                           mode='lines',
                           opacity=1,
                               line = dict(dash ='solid', color = coldict[model_m1], width=3),
                           showlegend=False), row=3, col=1)

    
    
    fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
                               y=model_dict['ScalingFactors'],
                               name= model,
                               mode='markers',
                               opacity=1,
#                                    marker=dict(color=coldict[model_m1], 
#                                                size=5,
#                                                symbol='line-ew',
#                                                line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
                                   marker=dict(color=coldict[model_m1], size=5),
                               line = dict(shape = 'hvh',dash ='dot', color = coldict[model_m1], width=1),
                               showlegend=False),
                               secondary_y=False,row=1, col=1)

    
    
    time_avg,dscale_avg = orbit_avg_generic(den_dict['dates'], den_dict['denscaled'], den_dict['lat'])
    ## -----------------------------------------------------------------------------------------------------
    ##     Orbit Averaged Density
    fig.add_trace(go.Scattergl(x=time_avg,
                               y=dscale_avg,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                                   marker=dict(color=coldict[model_m1],size=2),
                                   line = dict(dash ='solid', color = coldict[model_m1], width=2),
                               showlegend=False), row=2, col=1)

    
    
    
    
#     time_avg,d_avg = orbit_avg_generic(den_dict['dates'], den_dict['dens'], den_dict['lat'])
    ### -----------------------------------------------------------------------------------------------------

    ###     Orbit Averaged Density
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=d_avg,
#                                ### name= model_m1,
#                                mode='markers',
#                                opacity=0.3,
#                                    marker=dict(color= coldict[model_m1],size=2),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=2, col=1)
         
    return(fig)







fig  = make_subplots(
    rows=3, cols=1,
#     subplot_titles=(['Density (kg/m^3)'   , 
# #                      'CD*A'               ,
#                      'CD'                 ,   
#                      'Relative Velocity'  , 
#                      'Speed Ratio',
#                     ]),
    vertical_spacing = 0.05,
#     horizontal_spacing = 0.05,
    shared_xaxes=True)



# run_dict = { }
# rms_total_return = {}

for model in run_dict.keys():
    for imonth,month in enumerate(month_list):

        fig = PLOT_resids(fig, obj[model],   models_dens[model], run_dict[model])

    
#######################################################
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = True
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)

    fig.update_yaxes(title_text="Scaling Factor", 
                     exponentformat= 'power',row=1, col=1)

    yaxis_range = [-13.7 -.55 ,  -12.6+.25]# ] #full_fig.layout.yaxis2.range

    fig.update_yaxes(title_text="Scaled Density", 
                     type="log", 
                      range=yaxis_range,
                     exponentformat= 'power',row=2, col=1)
    fig.update_yaxes(title_text="CD*A", 
#                       type="log", 
#                       range=yaxis_range,
                     exponentformat= 'power',row=3, col=1)
    
a='input'
s=settings_icesat2

fig.update_layout(title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}",
#                     title=f"BWDRAG, 3-hr Scale from CD=2.5 ",
                  autosize=False,    width=1000,    height=1000,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title
fig.show(config=config)






In [ ]:
# %load_ext autoreload
# %autoreload 2


# def PLOT_resids(fig, obj_m1, den_dict, model_dict):
    
#     SHOW_alldata = True
    
#     model_m1 = obj_m1['global_params']['prms']['den_model']
#     if model!='hasdm_oc':
#         opac_val = 1
#     else:
#         opac_val = 1

#     for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
#         epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = obj_m1['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        

#         data_res = obj_m1['OrbitResids'][arc_name]['resids']

#         drag_data        = obj_m1['DragFile'][arc_name]
#         drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]
        
# #         print(arc[:8])
#         ### Cut off the extra time for the regular days
#         start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
#         end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

# #         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         drag_data = drag_data.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         den_df = den_df.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )


#         indplot=3

# #         date_rms.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) + datetime.timedelta(int(arc.split('.')[1]))- datetime.timedelta(hours=12) ))
# #         rms_totals.append(obj_m1['Statistics'][arc]['T_RMS'].values[0])

# #         iters = obj_m1['run_parameters'+arc_name]['total_iterations']
# #         for itime in obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD']:
# #             date_scalefactor.append(itime)
# #             ScalingFactor.append(obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD'][itime]['CURRENT_VALUE'])


#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'],
#                                        y=drag_data['CD']*drag_data['TOTAREA'],
#                                      mode='markers',
#                                      opacity=1,
#                                      marker=dict(color= coldict[model_m1],size=3,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=4, col=1,
#                                  )
# #         (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    
# #         fig.add_trace(go.Scattergl(x=time_avg,
# #                                  y=avg_cd,
# #                            mode='lines',
# #                            opacity=1,
# #                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
# #                            showlegend=False), row=3, col=1)

    
    
#     fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
#                            y=model_dict['ScalingFactors'],
#                            name= model,
#                            mode='markers+lines',
#                            opacity=1,
#                                marker=dict(color=coldict[model_m1], 
#                                            size=15,
#                                            symbol='line-ew',
#                                            line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
#                            line = dict(shape = 'hvh',dash ='dash', color = coldict[model_m1], width=1),
#                            showlegend=False),
#                            secondary_y=False,row=1, col=1)

    
    
#     time_avg,dscale_avg = orbit_avg_generic(den_dict['dates'], den_dict['denscaled'], den_dict['lat'])
#     ## -----------------------------------------------------------------------------------------------------
#     ##     Orbit Averaged Density
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=dscale_avg,
#                                ### name= model_m1,
#                                mode='markers+lines',
#                                opacity=1,
#                                    marker=dict(color=coldict[model_m1],size=2),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=3, col=1)
# #     fig.add_trace(go.Scattergl(x=den_dict['dates'],
# #                                y=den_dict['denscaled'],
# #                                ### name= model_m1,
# #                                mode='markers+lines',
# #                                opacity=1,
# #                                    marker=dict(color=coldict[model_m1],size=2),
# #                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
# #                                showlegend=False), row=3, col=1)

    
    
    
    
#     time_avg,d_avg = orbit_avg_generic(den_dict['dates'], den_dict['dens'], den_dict['lat'])
#     ### -----------------------------------------------------------------------------------------------------
#     ###     Orbit Averaged Density
# #     fig.add_trace(go.Scattergl(x=den_dict['dates'], #time_avg,
# #                                y=den_dict['dens'], #d_avg,
# #                                ### name= model_m1,
# #                                mode='markers+lines',
# #                                opacity=1,
# #                                    marker=dict(color=coldict[model_m1],size=1),
# #                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
# #                                showlegend=False), row=2, col=1)

#     ###     Orbit Averaged Density
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=d_avg,
#                                ### name= model_m1,
#                                mode='markers+lines',
#                                opacity=1,
#                                    marker=dict(color= coldict[model_m1],size=2),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=2, col=1)
    
    
# #                 time_avg,CD_avg, area_avg_rolling, CD_std = orb_avg_param(obj_m1.__dict__['DragFile'], arc_string, 'TOTAREA')
# #                 fig.add_trace(go.Scattergl(x=time_avg,
# #                                            y=area_avg_rolling,
# #     #                                          name= plot_name+arc,
# #                                              mode='markers+lines',
# #                                              opacity=1,
# #                                              marker=dict(color=col,size=2,),
# #                                              line = dict(shape='hvh', dash ='solid', color = col, width=3),
# #                                              showlegend=False, ),
# #                                              secondary_y=False,
# #                                              row=3, col=1,
# #                                          )



        

        
#     return(fig)







# fig  = make_subplots(
#     rows=4, cols=1,
# #     subplot_titles=(['Density (kg/m^3)'   , 
# # #                      'CD*A'               ,
# #                      'CD'                 ,   
# #                      'Relative Velocity'  , 
# #                      'Speed Ratio',
# #                     ]),
#     vertical_spacing = 0.05,
# #     horizontal_spacing = 0.05,
#     shared_xaxes=True)



# # run_dict = { }
# # rms_total_return = {}

# for model in run_dict.keys():
#     for imonth,month in enumerate(month_list):

# #     for ialph,alpha in enumerate(alphas):   
#          # fig,rms_total_return = PLOT__intrack_residuals_w_rms_w_cd(fig, Obj_Geodyn['msis2'] , 0,  arc_listlist)
# #         run_dict[val+"alpha_"+alpha] = ialph

#         fig = PLOT_resids(fig, obj[model],   models_dens[model], run_dict[monthmodel])

    
# #######################################################
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)

#     fig.update_yaxes(title_text="Scaling Factor", 
#                      exponentformat= 'power',row=1, col=1)

#     fig.update_yaxes(title_text="Density", 
#                      type="log", 
#                      exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes(title_text="Scaled Density", 
#                       type="log", 
#                      exponentformat= 'power',row=3, col=1)
# #     D
# a='input'
# s=settings_icesat2
# fig.update_layout(title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}", 
#                   autosize=False,    width=1000,    height=1100,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=16)  # Increase size of subplot title
# fig.show(config=config)




In [ ]:
# run_dict

## Plot Drag State

In [ ]:
# obj['jb2008']['DragFile']['2018.303']

In [ ]:
# arc = obj['jb2008']['global_params']['arc_input'][0]

# print(pd.to_datetime(arc, format = '%Y.%j'))
# print(pd.to_datetime(arc, format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm'))


In [ ]:
# %load_ext autoreload
# %autoreload 2


# def PLOT__SATDRAG_STATE(fig, obj_m1, den_dict):
    
#     SHOW_alldata = False
    
#     model_m1 = obj_m1['global_params']['prms']['den_model']
#     if model!='hasdm_oc':
#         opac_val = 1
#     else:
#         opac_val = 1

#     for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
#         epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = obj_m1['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        



#         drag_data        = obj_m1['DragFile'][arc_name]
#         drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]
        
# #         print(arc[:8])
#         ### Cut off the extra time for the regular days
#         start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
#         end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

# #         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         drag_data = drag_data.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         den_df = den_df.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )


#         indplot=3

# #         date_rms.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) + datetime.timedelta(int(arc.split('.')[1]))- datetime.timedelta(hours=12) ))
# #         rms_totals.append(obj_m1['Statistics'][arc]['T_RMS'].values[0])

# #         iters = obj_m1['run_parameters'+arc_name]['total_iterations']
# #         for itime in obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD']:
# #             date_scalefactor.append(itime)
# #             ScalingFactor.append(obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD'][itime]['CURRENT_VALUE'])


#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['CD'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=2, col=1,
#                                  )
#         (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=2, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['TOTAREA'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=3, col=1,
#                                  )
#         (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_area,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=3, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['CD'][::indplot]*drag_data['TOTAREA'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=4, col=1,
#                                  )
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd*avg_area,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=4, col=1)

        
#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['VELREL'][::indplot],
# #                                          name= plot_name+arc,
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
#                                      showlegend=False, ),
#                                      secondary_y=False,
#                                      row=5, col=1,
#                                  )
#         (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['VELREL'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=5, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                    y=drag_data['SpeedRatio'][::indplot],
# #                                          name= plot_name+arc,
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
#                                      showlegend=False, ),
#                                      secondary_y=False,
#                                      row=6, col=1,
#                                  )
#         (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['SpeedRatio'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=6, col=1)
#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=den_df['Date'][::indplot],
#                                  y=den_df['rho (kg/m**3)'][::indplot],
#                        ### name= model_m1,
#                        mode='markers',
#                        opacity=0.5,
#                            marker=dict(color=coldict[model_m1],size=2),
#                            line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                        showlegend=False), row=1, col=1)
        
#         (time_avg, d_avg ) = orbit_avg_generic(den_df['Date'],den_df['rho (kg/m**3)'],
#                                                den_df['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=d_avg,
#                            mode='lines',
#                            opacity=1,
#                                marker=dict(color=coldict[model_m1],size=1),
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=1, col=1)

    

    
    
# #     (time_avg, d_avg ) = orbit_avg_generic(den_dict['dates'],
# #                                    den_dict['den'],
# #                                    den_dict['lat'])
# #     ### -----------------------------------------------------------------------------------------------------
# #     ###     Orbit Averaged Density
# #     fig.add_trace(go.Scattergl(x=time_avg,
# #                                y=d_avg,
# #                                ### name= model_m1,
# #                                mode='markers',
# #                                opacity=opac_val,
# #                                    marker=dict(color= coldict[model_m1],size=2),
# #                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
# #                                showlegend=False), row=1, col=1)
    
    
# #                 time_avg,CD_avg, area_avg_rolling, CD_std = orb_avg_param(obj_m1.__dict__['DragFile'], arc_string, 'TOTAREA')
# #                 fig.add_trace(go.Scattergl(x=time_avg,
# #                                            y=area_avg_rolling,
# #     #                                          name= plot_name+arc,
# #                                              mode='markers+lines',
# #                                              opacity=1,
# #                                              marker=dict(color=col,size=2,),
# #                                              line = dict(shape='hvh', dash ='solid', color = col, width=3),
# #                                              showlegend=False, ),
# #                                              secondary_y=False,
# #                                              row=3, col=1,
# #                                          )



        

        
#     return(fig)







# fig  = make_subplots(
#     rows=6, cols=1,
# #     subplot_titles=(['Density (kg/m^3)'   , 
# # #                      'CD*A'               ,
# #                      'CD'                 ,   
# #                      'Relative Velocity'  , 
# #                      'Speed Ratio',
# #                     ]),
#     vertical_spacing = 0.05,
# #     horizontal_spacing = 0.05,
#     shared_xaxes=True)



# # run_dict = { }
# # rms_total_return = {}

# for model in run_dict.keys():
#     for imonth,month in enumerate(month_list):

# #     for ialph,alpha in enumerate(alphas):   
#          # fig,rms_total_return = PLOT__intrack_residuals_w_rms_w_cd(fig, Obj_Geodyn['msis2'] , 0,  arc_listlist)
# #         run_dict[val+"alpha_"+alpha] = ialph

#         fig = PLOT__SATDRAG_STATE(fig, obj[month+model],   models_dens[month+model])

    
# #######################################################
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)

    
#     fig.update_yaxes(title_text="Density", 
#                      type="log", 
#                      exponentformat= 'power',row=1, col=1)
#     fig.update_yaxes(title_text="CD", 
#                      exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes(title_text="Total Area", 
#                      exponentformat= 'power',row=3, col=1)
#     fig.update_yaxes(title_text="CD*Area", 
#                      exponentformat= 'power',row=4, col=1)

#     fig.update_yaxes(title_text="Rel. Velocity [m/s] ", 
#                      exponentformat= 'power',row=5, col=1)
#     fig.update_yaxes(title_text="Speed Ratio", 
#                      exponentformat= 'power',row=6, col=1)

#     ###
#     ###  DATE on Final x-Axis only
# #     fig.update_xaxes(title="Date",
# #                      row=4, col=1)
# #     fig.update_xaxes(title="Date",
# #                      row=5, col=2)




# fig.update_layout(
# #                   title = '',
#                   autosize=False,    width=1000,    height=1100,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=16)  # Increase size of subplot title



# # fig.add_vrect(x0="2018-10-16 23:00:00" ,
# #               x1="2018-10-17 00:24:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="2018-10-28 16:00:00" ,
# #               x1="2018-10-28 17:36:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="2018-10-29 10:18:00" ,
# #               x1="2018-10-29 11:42:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="" ,
# #               x1="" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="" ,
# #               x1="" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)

# fig.show(config=config)




# What is the average CD for each time period?